In [1]:
import ROOT
import subprocess

In [2]:

whizard_prefix = subprocess.run(['whizard-config', '--prefix'], capture_output=True, encoding='ascii').stdout.strip()
whizard_libs = f"{whizard_prefix}/lib/"
print(whizard_libs)

/cvmfs/sw.hsf.org/key4hep/releases/2025-05-29/x86_64-almalinux9-gcc14.2.0-opt/whizard/3.1.4-zjmc7r/lib/


In [3]:
ROOT.gSystem.AddDynamicPath(whizard_libs)
ROOT.gSystem.Load("libomega_core.so")
ROOT.gSystem.Load("libwhizard.so")
ROOT.gSystem.Load("libwhizard_main.so")
ROOT.gSystem.Load("libomega.so")
ROOT.gSystem.Load("whizard/cc20_ac_inclusive/.libs/default_lib.so")

0

In [4]:
ROOT.gInterpreter.Declare("#include \"test_whizard_object.h\"")

True

In [5]:
from model_parser import ModelParser
model_parser = ModelParser("SM_ac.mdl")

# add derivation of lz and kz according to lep parametrisation
model_parser.add_derived_parameter("lz", "la")
model_parser.add_derived_parameter("kz", "1.0 - (ka - 1.0) * sw**2/cw**2 + (g1z - 1.0)")

pars = {}
pars["nominal"] = model_parser.get_parameters_list()
alt_configs = {
    "alt1": {
        "g1z": 1.001,
        "ka": 1.000,
        "la": 0.000,
    },
    "alt2": {
        "g1z": 1.000,
        "ka": 1.001,
        "la": 0.000,
    },
    "alt3": {
        "g1z": 1.000,
        "ka": 1.000,
        "la": 0.001,
    }
}
for name, config in alt_configs.items():
    model_parser.set_parameters(config)
    pars[name] = model_parser.get_parameters_list()

print(pars)

{'nominal': [1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 246.21845810181634, 0.8819013863635865, 0.4714339240338821, 0.30795615429614365, 0.0, 0.0, 0.0, 132.50494581248503], 'alt1': [1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.001, 0.0, 0.0, 0.0, 0.0, 1.0, 1.001, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 1.0, 1.0, 5000.0, 0.0, 0.0, 0.0, 0.0, 246.21845810181634, 0.8819013863635865, 0.4714339240338821, 0.30795615429614365, 0.0, 0.0, 0.0, 132.50494581248503], 'alt2': [1.16639e-05, 91.1882, 80.419, 125.0, 0.1178, 0.000511, 0.1057, 1.777, 0.12, 1.25, 4.2, 174.0, 1.523, 2.443, 2.049, 0.004143, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 1.0, 0.0, 

In [6]:
omega_wrappers = {name: ROOT.OmegaWrapper(par) for name, par in pars.items()}
print(omega_wrappers)

{'nominal': <cppyy.gbl.OmegaWrapper object at 0xa521a00>, 'alt1': <cppyy.gbl.OmegaWrapper object at 0xa58e530>, 'alt2': <cppyy.gbl.OmegaWrapper object at 0xa51b9d0>, 'alt3': <cppyy.gbl.OmegaWrapper object at 0xa559200>}


In [7]:
omega_nom = omega_wrappers["nominal"]
omega_alt1 = omega_wrappers["alt1"]

In [8]:
print(type(omega_nom))

<class cppyy.gbl.OmegaWrapper at 0x92eb700>


In [9]:
df = ROOT.RDataFrame("events", "whizard/cc20_ac_exclusive/ww.1k.edm4hep.root")

In [10]:

df = df.Define("mc_lvec", "Construct<ROOT::Math::PxPyPzMVector>(MCParticle.momentum.x, MCParticle.momentum.y, MCParticle.momentum.z, MCParticle.mass)")
df = df.Define("E", "return Map(mc_lvec, [] (const auto& el) {return el.energy();})")
df = df.Define("PX", "MCParticle.momentum.x")
df = df.Define("PY", "MCParticle.momentum.y")
df = df.Define("PZ", "MCParticle.momentum.z")

In [11]:

df = df.Define("momenta", """
               std::vector<double>({
               E[0], PX[0], PY[0], PZ[0],
               E[1], PX[1], PY[1], PZ[1],
               E[2], PX[2], PY[2], PZ[2],
               E[3], PX[3], PY[3], PZ[3],
               E[4], PX[4], PY[4], PZ[4],
               E[5], PX[5], PY[5], PZ[5],
               })
               """)
# XXX: does the calculation twice :(

In [12]:
ROOT.gInterpreter.Declare("#include <podio/GenericParameters.h>")
df = df.Define("Parameters", "podio::GenericParameters par; par.loadFrom(GPDoubleKeys, GPDoubleValues); par.loadFrom(GPFloatKeys, GPFloatValues); par.loadFrom(GPIntKeys, GPIntValues); par.loadFrom(GPStringKeys, GPStringValues); return par;")
df = df.Define("sqme_whizard", "Parameters.get<double>(\"sqme\").value_or(-42.0)")
df = df.Define("sqme_alt1_whizard", "Parameters.get<double>(\"sqme_alt1\").value_or(-42.0)")
df = df.Define("weight_alt1_whizard", "Parameters.get<double>(\"weight_alt1\").value_or(-42.0)")
df = df.Define("e_charge", "MCParticle.charge[2]")
df = df.Define("flv", "e_charge > 0 ? 1 : 2")

In [13]:
# df = df.Define("sqme", ROOT.OmegaWrapperFunctor(omega_nom), ["momenta", "flv"])
df = df.Define("sqme", omega_nom, ["momenta", "flv"])
df = df.Define("sqme_alt1", omega_alt1, ["momenta", "flv"])

In [14]:
df = df.Define("recalc_weight", "sqme / sqme_whizard")
df = df.Define("weight_alt1", "sqme_alt1 / sqme")

In [15]:
df.Display(["sqme_whizard", "sqme", "sqme_alt1", "sqme_alt1_whizard", "weight_alt1", "weight_alt1_whizard"]).Print()

+-----+--------------+----------+-----------+-------------------+-------------+---------------------+
| Row | sqme_whizard | sqme     | sqme_alt1 | sqme_alt1_whizard | weight_alt1 | weight_alt1_whizard | 
+-----+--------------+----------+-----------+-------------------+-------------+---------------------+
| 0   | 0.000208     | 0.000208 | 0.000208  | 0.000208          | 1.000429    | 1.000429            | 
+-----+--------------+----------+-----------+-------------------+-------------+---------------------+
| 1   | 0.004474     | 0.004474 | 0.004476  | 0.004476          | 1.000312    | 1.000312            | 
+-----+--------------+----------+-----------+-------------------+-------------+---------------------+
| 2   | 0.000000     | 0.000000 | 0.000000  | 0.000000          | 0.999594    | 0.999594            | 
+-----+--------------+----------+-----------+-------------------+-------------+---------------------+
| 3   | 0.000327     | 0.000327 | 0.000327  | 0.000327          | 1.000964    